In [2]:
import pandas as pd
import re
import os

In [3]:
stop_times = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs"+ os.sep + "stop_times.txt", delimiter=",")
stops = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs" + os.sep +  "stops.txt", delimiter=",")
routes = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs"+ os.sep + "routes.txt", delimiter=",")
trips = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs"+ os.sep + "trips.txt", delimiter=",")
calendar = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs"+ os.sep + "calendar.txt", delimiter=",")

In [4]:
stop_times["arrival_time"] = pd.to_timedelta(stop_times["arrival_time"])

In [10]:
departure = "00:31:41"
destination = "UCD Belfield, stop 768"
origin = "Dawson Street, stop 792"
day = "Saturday"
day = day.lower()
bus_route = "39A"
bus_route = bus_route.lower()

In [13]:
def get_route(departure, origin, destination, day, bus_route):
    day = day.lower()
    bus_route = bus_route.lower()
    days = list(calendar[calendar[day] == 1]["service_id"].values)
    day_trips = set(trips[trips["service_id"].isin(days)]["trip_id"].values)
    route_trips = set(
        trips[trips["route_id"].isin(list(routes[routes["route_short_name"] == bus_route]["route_id"].values))][
            "trip_id"].values)
    start_trips = stops[stops["stop_name"] == origin]["stop_id"].values[0]
    end_trips = stops[stops["stop_name"] == destination]["stop_id"].values[0]
    start = set(stop_times[stop_times["stop_id"] == start_trips]["trip_id"].values)
    end = set(stop_times[stop_times["stop_id"] == end_trips]["trip_id"].values)

    r = start.intersection(end)
    print(route_trips)
    r = r.intersection(route_trips)
    r = r.intersection(day_trips)
    I = None
    O = None
    for trip in r:
        if re.search(".I$", trip):
            I = trip
            break

    for trip in r:
        if re.search(".O$", trip):
            O = trip
            break
    print(O)
    print(I)
    correct_direction = []
    if O != None and (
            (stop_times[(stop_times["trip_id"] == O) & (stop_times["stop_id"] == start_trips)]["stop_sequence"].values[
                0]) < (
                    stop_times[(stop_times["trip_id"] == O) & (stop_times["stop_id"] == end_trips)][
                        "stop_sequence"].values[0])):
        correct_direction = [trip for trip in r if re.search(".O$", trip)]
    elif I != None and (
            (stop_times[(stop_times["trip_id"] == I) & (stop_times["stop_id"] == start_trips)]["stop_sequence"].values[
                0]) < (
                    stop_times[(stop_times["trip_id"] == I) & (stop_times["stop_id"] == end_trips)][
                        "stop_sequence"].values[0])):
        correct_direction = [trip for trip in r if re.search(".I$", trip)]
    final = stop_times[(stop_times["trip_id"].isin(correct_direction)) & (stop_times["stop_id"] == start_trips)][
        ["trip_id", "arrival_time"]]
    final["arrival_time"] = abs(final["arrival_time"] - pd.to_timedelta(departure))
    final = final.sort_values(by="arrival_time")
    tripid = final.iloc[0]["trip_id"]
    seq = stop_times[stop_times["trip_id"] == tripid][["stop_id", "stop_sequence"]].sort_values(by="stop_sequence")
    rstart = seq[seq["stop_id"] == start_trips].index[0]
    rfin = seq[seq["stop_id"] == end_trips].index[0]
    test = seq.loc[rstart:rfin]
    final = test.merge(stops, how='left', on='stop_id')
    flist = list(final["stop_name"].values)
    flist = [stop.split(" stop ")[1] for stop in flist]
    final_routes = []
    for i in range(len(flist) - 1):
        final_routes.append(flist[i] + "_" + flist[i+1])
    final_routes
    return final_routes

In [14]:
get_route(departure, origin, destination, day, bus_route)

{'5997.y1007.60-39A-d12-1.194.O', '6026.y1007.60-39A-d12-1.195.I', '11683.y1009.60-39A-d12-1.194.O', '5841.y1003.60-39A-b12-1.195.O', '6068.y1003.60-39A-b12-1.195.O', '11690.y1009.60-39A-d12-1.195.I', '15869.y1004.60-39A-b12-1.195.O', '11570.y1005.60-39A-b12-1.196.I', '11724.y1005.60-39A-b12-1.195.O', '15891.y1004.60-39A-b12-1.195.O', '5999.y1007.60-39A-d12-1.194.O', '15938.y1008.60-39A-d12-1.194.O', '2517.y1003.60-39A-b12-1.195.O', '6073.y1007.60-39A-d12-1.195.I', '5979.y1007.60-39A-d12-1.194.O', '15951.y1008.60-39A-d12-1.194.O', '6042.y1007.60-39A-d12-1.195.I', '6389.y1007.60-39A-d12-1.194.O', '11617.y1005.60-39A-b12-1.195.O', '11581.y1009.60-39A-d12-1.195.I', '15828.y1004.60-39A-b12-1.195.O', '11771.y1009.60-39A-d12-1.194.O', '6085.y1003.60-39A-b12-1.195.O', '15978.y1004.60-39A-b12-1.195.O', '11650.y1005.60-39A-b12-1.195.O', '6020.y1007.60-39A-d12-1.194.O', '5971.y1007.60-39A-d12-1.194.O', '11778.y1009.60-39A-d12-1.194.O', '11550.y1005.60-39A-b12-1.195.O', '11767.y1005.60-39A-b12-1.

IndexError: single positional indexer is out-of-bounds

In [187]:
flist = ['404',
 '747',
 '748',
 '749',
 '750',
 '751',
 '752',
 '753',
 '754',
 '755',
 '756',
 '757',
 '758',
 '759',
 '760',
 '761',
 '762',
 '763',
 '767']

In [190]:
len(flist)

19

In [191]:
final_routes = []
for i in range(len(flist) - 1):
    final_routes.append(flist[i] + "_" + flist[i+1])
final_routes

['404_747',
 '747_748',
 '748_749',
 '749_750',
 '750_751',
 '751_752',
 '752_753',
 '753_754',
 '754_755',
 '755_756',
 '756_757',
 '757_758',
 '758_759',
 '759_760',
 '760_761',
 '761_762',
 '762_763',
 '763_767']

In [ ]:
test

In [171]:
days = list(calendar[calendar[day] == 1]["service_id"].values)

In [172]:
day_trips = set(trips[trips["service_id"].isin(days)]["trip_id"].values)

In [173]:
route_trips = set(trips[trips["route_id"].isin(list(routes[routes["route_short_name"] == bus_route]["route_id"].values))]["trip_id"].values)

In [174]:
start_trips = stops[stops["stop_name"] == origin]["stop_id"].values[0]
end_trips = stops[stops["stop_name"] == destination]["stop_id"].values[0]

In [176]:
start  = set(stop_times[stop_times["stop_id"] == start_trips]["trip_id"].values)
end  = set(stop_times[stop_times["stop_id"] == end_trips]["trip_id"].values)

In [179]:
r = start.intersection(end)
r = r.intersection(route_trips)
r = r.intersection(day_trips)

In [180]:
I = None
O = None

for trip in r:
    if re.search(".I$", trip):
        I = trip
        break
        
for trip in r:
    if re.search(".O$", trip):
        O = trip
        break
print(O)
print(I)

None
2503.y1003.60-39A-b12-1.196.I


In [181]:
if O != None and ((stop_times[(stop_times["trip_id"] == O) & (stop_times["stop_id"] == start)]["stop_sequence"].values[0]) < (stop_times[(stop_times["trip_id"] == O) & (stop_times["stop_id"] == end)]["stop_sequence"].values[0])):
    correct_direction = [trip for trip in r if re.search(".O$", trip)]
elif I != None and ((stop_times[(stop_times["trip_id"] == I) & (stop_times["stop_id"] == start)]["stop_sequence"].values[0]) < (stop_times[(stop_times["trip_id"] == I) & (stop_times["stop_id"] == end)]["stop_sequence"].values[0])):
    correct_direction = [trip for trip in r if re.search(".I$", trip)]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
final = stop_times[(stop_times["trip_id"].isin(correct_direction)) & (stop_times["stop_id"] == start)][["trip_id","arrival_time"]]
final["arrival_time"] = abs(final["arrival_time"] - pd.to_timedelta(departure))
final = final.sort_values(by="arrival_time")

In [ ]:
final

In [ ]:
tripid = final.iloc[0]["trip_id"]

In [ ]:
seq = stop_times[stop_times["trip_id"] == tripid][["stop_id", "stop_sequence"]].sort_values(by="stop_sequence")

In [ ]:
rstart = seq[seq["stop_id"] == start].index[0]

In [ ]:
rfin = seq[seq["stop_id"] == end].index[0]

In [ ]:
test = seq.loc[rstart:rfin]

In [ ]:
final = test.merge(stops, how='left', on='stop_id')

In [ ]:
flist = list(final["stop_name"].values)
flist = [stop.split(" stop ")[1] for stop in flist]
flist

In [192]:
from datetime import datetime

In [204]:
datetime.fromtimestamp(1.62639e+18/ 1e9).strftime("%H")

'00'

In [205]:
"00:14:40".split(":")[1]

'14'